In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from dateutil.relativedelta import relativedelta
from scipy.stats import ttest_rel 

In [2]:
## Reading data
RAW_COUNTIES_FILENAME = 'us-counties.csv'
POLICIES_FILENAME = 'earliestpolicy_08042020.csv'
raw_counties = pd.read_csv(RAW_COUNTIES_FILENAME)
raw_policies = pd.read_csv(POLICIES_FILENAME)[['fips','policy_date']]
raw_counties.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0


In [3]:
## Calculating R0 values
raw_counties['date'] = pd.to_datetime(raw_counties['date'])
sorted_counties = raw_counties.sort_values(by=['county','state','date'])
sorted_counties['new_cases'] = sorted_counties['cases'].diff() 
diff_mask = (sorted_counties['county'] != sorted_counties['county'].shift(1)) & (sorted_counties['state'] != sorted_counties['state'].shift(1))
sorted_counties['new_cases'] = sorted_counties['new_cases'].where(~diff_mask, np.nan)
sorted_counties['R0']= sorted_counties.new_cases / sorted_counties.new_cases.shift()
sorted_counties = sorted_counties.drop(columns=['deaths','cases'])

In [4]:
## Making fips to policy date dictionary
policy_dates = [dt.datetime.strptime(date,'%m-%d-%Y') if isinstance(date,str) else None for date in  raw_policies.policy_date ]
fips_2_dates = dict(zip(raw_policies.fips,policy_dates))
sorted_counties['policy_date']= sorted_counties['fips'].map(fips_2_dates)

In [5]:
## Creating before and after mandate dataframes
sorted_counties = sorted_counties.dropna()
after_mandate = sorted_counties[sorted_counties.apply(lambda x: x['date'] > x['policy_date'] + relativedelta(days=+14) and x['date'] < x['policy_date'] + relativedelta(days=+44), axis=1)] \
.replace([np.inf, -np.inf], np.nan).dropna()
before_mandate = sorted_counties[sorted_counties.apply(lambda x: x['date'] < x['policy_date'] and x['date'] > x['policy_date'] + relativedelta(days=-30), axis=1)] \
.replace([np.inf, -np.inf], np.nan).dropna()


In [6]:
print(before_mandate)
print(after_mandate)

             date     county           state     fips  new_cases        R0  \
94585  2020-04-27  Abbeville  South Carolina  45001.0        2.0  0.666667   
97424  2020-04-28  Abbeville  South Carolina  45001.0        0.0  0.000000   
105989 2020-05-01  Abbeville  South Carolina  45001.0        0.0  0.000000   
114612 2020-05-04  Abbeville  South Carolina  45001.0        0.0  0.000000   
126491 2020-05-08  Abbeville  South Carolina  45001.0        0.0  0.000000   
...           ...        ...             ...      ...        ...       ...   
274148 2020-06-25     Zavala           Texas  48507.0        1.0  0.500000   
277297 2020-06-26     Zavala           Texas  48507.0        1.0  1.000000   
280446 2020-06-27     Zavala           Texas  48507.0        1.0  1.000000   
283594 2020-06-28     Zavala           Texas  48507.0        0.0  0.000000   
293048 2020-07-01     Zavala           Texas  48507.0        0.0  0.000000   

       policy_date  
94585   2020-05-26  
97424   2020-05-26  


In [16]:
after_mandate_avg = after_mandate.drop(columns=['date','policy_date']).groupby(['state','county','fips']).mean()
before_mandate_avg = before_mandate.drop(columns=['date','policy_date']).groupby(['state','county','fips']).mean()
print(len(after_mandate_avg))
print(len(before_mandate_avg))

2039
1989


In [36]:
condensed = pd.merge(before_mandate_avg,after_mandate_avg,how='inner',on=['state','county','fips'],suffixes=('_b','_a')).dropna()
pd.options.display.max_colwidth = 500
condensed = condensed[(condensed['new_cases_b'] > 40) & (condensed['new_cases_a'] > 40)]

In [37]:
print(condensed)

                                   new_cases_b      R0_b  new_cases_a  \
state      county         fips                                          
Alabama    Jefferson      1073.0     67.206897  1.331326   226.275862   
           Mobile         1097.0     47.586207  2.408474   188.482759   
           Montgomery     1101.0     72.655172  1.143797    75.689655   
           Tuscaloosa     1125.0     40.103448  1.632649    57.206897   
Arizona    Maricopa       4013.0    626.344828  1.383642  2084.482759   
...                                        ...       ...          ...   
Virginia   Prince William 51153.0   123.392857  1.114370    45.862069   
Washington King           53033.0    83.344828  0.884806    69.103448   
           Yakima         53077.0   118.583333  1.000282    78.346154   
Wisconsin  Dane           55025.0    71.448276  1.381387    44.517241   
           Milwaukee      55079.0   154.827586  1.107440   175.655172   

                                       R0_a  
stat

In [38]:
ttest_rel(condensed['R0_b'], condensed['R0_a'])

Ttest_relResult(statistic=3.7065599896479533, pvalue=0.00030642543493743994)

In [39]:
print(condensed[['R0_b','R0_a']].mean())

R0_b    1.373674
R0_a    1.157741
dtype: float64
